In [1]:
import csv
import cv2
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda,Dropout,Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [3]:
# Change to locate path and Read image 
def read_image(source_path):
    filename = source_path.split('/')[-1]
    current_path = '/home/yue/CarND-Behavioral-Cloning/Training_data3/IMG/' + filename
    image = cv2.imread(current_path)
    converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return converted_image


# Open csv file
lines = []
with open('/home/yue/CarND-Behavioral-Cloning/Training_data3/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [4]:
center_image_paths = []
center_steers = []

left_image_paths = []
left_steers = []

right_image_paths = []
right_steers = []

image_paths = []
steers = [] 

for line in lines:
    #center
    center_image_paths.append(line[0])
    center_steers.append(float(line[3]))
    
    #left
    left_image_paths.append(line[1])
    left_steers.append(float(line[3])+0.25)
    
    #right
    right_image_paths.append(line[2])
    right_steers.append(float(line[3])-0.25)
    
    image_paths.append(line[0])
    steers.append(float(line[3]))
    
    #left
    image_paths.append(line[1])
    steers.append(float(line[3])+0.25)
    
    #right
    image_paths.append(line[2])
    steers.append(float(line[3])-0.25)
    

center_image_paths = np.array(center_image_paths)
center_steers = np.array(center_steers)

left_image_paths = np.array(left_image_paths)
left_steers = np.array(left_steers)

right_image_paths = np.array(right_image_paths)
right_steers = np.array(right_steers)

image_paths = np.array(image_paths)
steers = np.array(steers)

In [5]:
def data_distribution_normalize(image_paths, steers): 
    num_bins = 50
    n, bins = np.histogram(steers, num_bins)
    
    keep_probs = []
    target = len(steers)/(num_bins)

    remove_list = []

    for i in range(num_bins):
        if n[i] < target:
            keep_probs.append(1.)
        else:
            keep_probs.append(target/n[i])

    for i in range(len(steers)):
        for j in range(num_bins):
            if steers[i] > bins[j] and steers[i] <= bins[j+1]:
                if np.random.rand() > keep_probs[j]:
                    remove_list.append(i)
                    
    image_paths = np.delete(image_paths, remove_list)
    steers = np.delete(steers, remove_list)
    
    return image_paths, steers


'''
center_image_paths, center_steers = data_distribution_normalize(center_image_paths, center_steers)
left_image_paths, left_steers = data_distribution_normalize(left_image_paths, left_steers)
right_image_paths, right_steers = data_distribution_normalize(right_image_paths, right_steers)
#plt.hist(right_steers, 50)
dist = []
dist.append(len(center_image_paths))
dist.append(len(left_image_paths))
dist.append(len(right_image_paths))
for num in dist:
    print(num)


image_paths = np.concatenate((center_image_paths,left_image_paths,right_image_paths))
steers = np.concatenate((center_steers,left_steers,right_steers))
print(len(image_paths))
'''    
new_image_paths, new_steers = data_distribution_normalize(image_paths, steers)
print(len(new_image_paths))
print(len(new_steers))

10504
10504


In [6]:
# split dataset to 80% training data, 20% validation data
train_samples_paths, validation_samples_paths, train_steers, validation_steers = train_test_split(new_image_paths, new_steers,test_size = 0.2)
print(len(train_samples_paths))

print(len(validation_samples_paths))


8403
2101


In [7]:
def translation_image(image, steer,trans_range):
    rand_num = np.random.uniform(-trans_range,trans_range)
    rand_num2 = np.random.uniform(-trans_range,trans_range)
    rows, cols, ch = image.shape
    h_shift = rand_num
    trans_steer = steer + h_shift * 0.004
    v_shift = rand_num2
    M = np.float32([[1,0,h_shift],[0,1,v_shift]])
    trans_image = cv2.warpAffine(image,M,(cols,rows))
    
    return trans_image, trans_steer

In [15]:
def random_shadow(image):
    new_img = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    new_img = np.array(new_img, dtype = np.float64)

    h,w = new_img.shape[0:2]
    mid = np.random.randint(0,w)


    factor = np.random.uniform(0.6,0.8)
    if np.random.rand() > .5:
        new_img[:,0:mid,0] *= factor
    else:
        new_img[:,mid:w,0] *= factor

    shadow_image = np.array(new_img, dtype = np.uint8)  
    #shadow_image = cv2.cvtColor(shadow_image,cv2.COLOR_HSV2RGB)

    return shadow_image


In [16]:
def brightness_image(image):
    image_tmp = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    image_tmp = np.array(image_tmp, dtype = np.float64)
    random_bright = .5+np.random.uniform()
    image_tmp[:,:,0] = image_tmp[:,:,2]*random_bright
    image_tmp[:,:,0][image_tmp[:,:,2]>255]  = 255
    image_tmp = np.array(image_tmp, dtype = np.uint8)
    #image_tmp = cv2.cvtColor(image_tmp,cv2.COLOR_HSV2RGB)
    
    return image_tmp


In [17]:
def data_augmentation(image_paths,steers, aug_num):
    new_images = []
    new_measurements = []
    for i in range(len(image_paths)):
        image = read_image(image_paths[i])
        steer = steers[i]
        new_images.append(image)
        new_measurements.append(steer)
        flip_image = np.fliplr(image)
        new_images.append(flip_image)
        new_measurements.append(-steer)

        for i in range(aug_num):
            new_image = brightness_image(image)
            new_image = random_shadow(new_image)
            new_image, new_steer = translation_image(new_image, steer,30) 
            new_images.append(new_image)
            new_measurements.append(new_steer)

            new_flip_image = brightness_image(flip_image)
            new_flip_image = random_shadow(new_flip_image)
            new_flip_image, new_flip_steer = translation_image(new_flip_image, -steer,30) 
            new_images.append(new_flip_image)
            new_measurements.append(new_flip_steer)
    
    return np.array(new_images), np.array(new_measurements)
    
    
    

In [18]:
# Generator with coroutine
def train_generator(samples, steers, batch_size = 32):
    num_samples = len(samples)
    while 1:
        # shuffle data
        sklearn.utils.shuffle(samples,steers)
        #Loop over batches
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            batch_steers = steers[offset:offset+batch_size]
                    
            images, measurements = data_augmentation(batch_samples, batch_steers,1)

            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

In [19]:
# Generator with coroutine
def valid_generator(samples, steers, batch_size = 32):
    num_samples = len(samples)
    while 1:
        # shuffle data
        sklearn.utils.shuffle(samples, steers)
        #Loop over batches
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            batch_steers = steers[offset:offset+batch_size]

            images = []
            measurements = []
            for i in range(len(batch_samples)):

                images.append(read_image(batch_samples[i])) 
                
                measurements.append(float(steers[i]))

            X_valid = np.array(images)
            y_valid = np.array(measurements)
            yield sklearn.utils.shuffle(X_valid, y_valid)

In [20]:
batch_size = 64

training_generator = train_generator(train_samples_paths, train_steers, batch_size=batch_size)
validation_generator = valid_generator(validation_samples_paths, validation_steers, batch_size=batch_size)


In [22]:
model = Sequential()
#Normalize the data
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
#Crop images to 80X320X3
model.add(Cropping2D(cropping=((60,20),(0,0))))
#Nvidia Network
model.add(Convolution2D(24,5,5,subsample=(2,2),activation = 'relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation = 'relu'))
model.add(Convolution2D(48,3,3,subsample=(2,2),activation = 'relu'))
model.add(Convolution2D(64,3,3,subsample=(1,1),activation = 'relu'))
model.add(Convolution2D(64,3,3,subsample=(1,1),activation = 'relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1))

#Optimize MSE using ADAM optimizer
model.compile(loss='mse', optimizer='adam')
train_steps = np.ceil(len(train_samples_paths)*4/batch_size).astype(np.int32)
validation_steps =np.ceil(len(validation_samples_paths)/batch_size).astype(np.int32)

model.fit_generator(training_generator, samples_per_epoch=len(train_samples_paths)*4, validation_data=validation_generator, nb_val_samples=len(validation_samples_paths), nb_epoch = 5)


model.save('model.h5')

/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  import sys
/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  
/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), activation="relu", strides=(2, 2))`
  if __name__ == '__main__':
/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", strides=(1, 1))`
  # Remove the CWD from sys.path while we load stuff.
/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launc

Epoch 1/5
   56/33612 [..............................] - ETA: 4:31:49 - loss: 0.0574

KeyboardInterrupt: 